# import library

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)

from bs4 import BeautifulSoup
import requests
import json

import re

from pprint import pprint

from keys import *

# scrape from CRLA HP

In [2]:
crla_url = "https://www.crla.org/office-locations"

crla_html = requests.get(crla_url).text

crla_soup = BeautifulSoup(crla_html)

In [13]:
crla_div = crla_soup.find_all("div", class_="row")[4]
crla_div = crla_div.find_all("div")

crla_office_dict = {"name": ["Administrative Office".lower()],
                    "address": ["1430 Franklin Street, Suite 103, Oakland, CA 94612".lower()]}

for div in crla_div:
    name = div.find("strong").text.lower()
    
    br = div.find_all("br")
    
    address = []
    
    for row in br:
        row_text = str(row.next_element)
        row_text = row_text.strip().replace("\"", "").replace("#", "").lower()
        if "admini" in row_text or "intake" in row_text or "(" in row_text or "<a href" in row_text:
                continue
        else:
            address.append(row_text)
   
    address = ", ".join(address)

    crla_office_dict["name"].append(name)
    crla_office_dict["address"].append(address)
    
crla_office_df = pd.DataFrame(crla_office_dict)
crla_office_df.head()

,name,address
0,administrative office,"1430 franklin street, suite 103, oakland, ca 9..."
1,arvin,"211 bear mountain blvd., arvin, ca 93203"
2,coachella,"1460 6th street, p.o. box 35, coachella, ca 92236"
3,delano,"601 high street, suite c, delano, ca 93215"
4,el centro,"449 broadway st., el centro, ca 92243"


# geocoding

In [14]:
endpoint = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'

params = {'limit': 1,
          'access_token': mapbox_access_token}

latitude = []
longitude = []

for idx, add in enumerate(crla_office_df["address"]):

    url = requests.Request('GET', endpoint+add+'.json', params=params).prepare().url
    response = requests.get(url)
    results = response.text
    data = json.loads(results)

    longitude.append(data["features"][0]["center"][0])
    latitude.append(data["features"][0]["center"][1])

crla_office_df["longitude"] = longitude
crla_office_df["latitude"] = latitude
crla_office_df.head()

,name,address,longitude,latitude
0,administrative office,"1430 franklin street, suite 103, oakland, ca 9...",-122.26919,37.804375
1,arvin,"211 bear mountain blvd., arvin, ca 93203",-118.82564,35.208841
2,coachella,"1460 6th street, p.o. box 35, coachella, ca 92236",-116.17685,33.678303
3,delano,"601 high street, suite c, delano, ca 93215",-119.24624,35.763884
4,el centro,"449 broadway st., el centro, ca 92243",-115.55365,32.793576


In [1]:
crla_office_df.to_csv("../data/resources/crla_office_locations.csv")

NameError: name 'crla_office_df' is not defined